<a href="https://colab.research.google.com/github/namwootree/Algorithm_study/blob/main/Aphaco_(Deep_Learning_Boot_Camp)/Long-Term%20Program/Text_Classification/%5BMain%5D_StandardKFold_(5)_Ensemble_(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setting

In [1]:
!pip install cloud-tpu-client==0.10 torch==1.11.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.11-cp37-cp37m-linux_x86_64.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 152.9 MB 45 kB/s 
     |████████████████████████████████| 57 kB 3.1 MB/s 
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.11
    Uninstalling google-api-python-client-1.12.11:
      Successfully uninstalled google-api-python-client-1.12.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.311 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 1.8.0 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -q folium
!pip install -q transformers
!pip install -q datasets
!pip install -q huggingface_hub

     |████████████████████████████████| 4.2 MB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 87.0 MB/s 
     |████████████████████████████████| 86 kB 4.3 MB/s 
     |████████████████████████████████| 596 kB 71.7 MB/s 
     |████████████████████████████████| 346 kB 5.2 MB/s 
     |████████████████████████████████| 1.1 MB 91.1 MB/s 
     |████████████████████████████████| 86 kB 6.6 MB/s 
     |████████████████████████████████| 140 kB 100.7 MB/s 
     |████████████████████████████████| 212 kB 101.1 MB/s 
     |████████████████████████████████| 127 kB 98.1 MB/s 
     |████████████████████████████████| 271 kB 96.2 MB/s 
     |████████████████████████████████| 144 kB 96.0 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
     |████████████████████████████████| 112 kB 93.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1

In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding
import datasets

from huggingface_hub import notebook_login

import torch
import torchvision
import torch_xla
import torch_xla.core.xla_model as xm

from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold

import pandas as pd
import numpy as np

import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

import gc

import tensorflow as tf

In [5]:
VERSION = "20220315"
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6034  100  6034    0     0  48661      0 --:--:-- --:--:-- --:--:-- 48661
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20220315 ...
Found existing installation: torch 1.11.0+cu113
Uninstalling torch-1.11.0+cu113:
Done updating TPU runtime
  Successfully uninstalled torch-1.11.0+cu113
Found existing installation: torchvision 0.12.0+cu113
Uninstalling torchvision-0.12.0+cu113:
  Successfully uninstalled torchvision-0.12.0+cu113
Copying gs://tpu-pytorch/wheels/colab/torch-nightly+20220315-cp37-cp37m-linux_x86_64.whl...
- [1 files][ 99.7 MiB/ 99.7 MiB]                                                
Operation completed over 1 objects/99.7 MiB.                                     
Copying gs://tpu-pytorch/wheels/colab/torch_xla-nightly+20220315-cp37-cp37m-linux_x86_64.whl...
\
Operation completed 

### Load Data

In [6]:
train_data_path = '/content/drive/MyDrive/알파코/프로젝트/장기 프로젝트/revised_data.csv'
test_data_path = '/content/drive/MyDrive/알파코/프로젝트/장기 프로젝트/test_data.csv'

In [7]:
df_train = pd.read_csv(train_data_path, index_col=False)
df_test = pd.read_csv(test_data_path, index_col=False)

display(df_train.head(), df_test.head())

,Unnamed: 0,title,topic_idx
0,0,"훌륭한 이름도 없고, 훌륭한 이름도 없고, 훌륭한 이름도 없다.",3
1,1,내일날씨 전국 구름 많음…아침 기온 다소 올라,3
2,2,김연경 금환...이제 도쿄 올림픽 출발을 위해 완성,5
3,3,브라질 최대정당 호세프와 결별할 듯…탄핵 가능성 커져,4
4,4,책·TV·PC 보면서도 스마트폰 손에서 못 뗀다,3


,index,title
0,45654,유튜브 내달 2일까지 크리에이터 지원 공간 운영
1,45655,어버이날 맑다가 흐려져…남부지방 옅은 황사
2,45656,내년부터 국가RD 평가 때 논문건수는 반영 않는다
3,45657,김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것
4,45658,회색인간 작가 김동식 양심고백 등 새 소설집 2권 출간


### Global Configration

In [8]:
num_labels = len(df_train['topic_idx'].unique())
n_splits = 5
models = ['klue/roberta-base', 'bert-base-multilingual-uncased', 'xlm-roberta-base']
batch_size = 32
num_train_epochs = 10

### Function Declaration

In [9]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [10]:
def tokenize(batch):
    return tokenizer(batch["title"], padding=True, truncation=True)

### Fine Tuning and Model Saving

In [11]:
for model_n in models:
    # Load Model & Tokenizer
    model_ckpt = model_n

    # Load Data & Prepare K-Fold

    X, y = df_train['title'], df_train['topic_idx']
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=100)
    splitgen = skf.split(X, y)

    print()
    print('----------------------------------------------------------------------------')
    print(model_ckpt)

    foldNum = 1
    for train_index, val_index in splitgen:
        print()
        print('----------------------------------------------------------------------------')
        print(f'{model_ckpt} {foldNum}-Fold')

        # Model instantiation

        model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels)
        device = xm.xla_device()
        model = model.to(device)

        tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
        data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

        # Tokenization
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        train = {
            'title': X_train.to_list(),
            'topic_idx': y_train.to_list()
        }
        
        val =  {
            'title': X_val.to_list(),
            'topic_idx': y_val.to_list()
        }
        
        train = datasets.Dataset.from_dict(train)
        train_encoded = train.map(tokenize, batched=True, batch_size=None)
        train_encoded = train_encoded.rename_column('topic_idx', 'label')

        val = datasets.Dataset.from_dict(val)
        val_encoded = val.map(tokenize, batched=True, batch_size=None)
        val_encoded = val_encoded.rename_column('topic_idx', 'label')

        # Training 
        logging_steps = len(train_encoded) // batch_size
        
        model_ckpt = model_ckpt.replace('/', '-')
        model_name = f"{model_ckpt}-tc-{foldNum}"

        training_args = TrainingArguments(output_dir='log/' + model_name,
                                            num_train_epochs=num_train_epochs,
                                            learning_rate=2e-5,
                                            per_device_train_batch_size=batch_size,
                                            per_device_eval_batch_size=batch_size,
                                            weight_decay=0.01,
                                            evaluation_strategy="epoch",
                                            disable_tqdm=False,
                                            logging_steps=logging_steps,
                                            push_to_hub=False,
                                            log_level="error",
                                            save_strategy='no')

        trainer = Trainer(model=model,
                    args=training_args,
                    compute_metrics=compute_metrics,
                    train_dataset=train_encoded,
                    eval_dataset=val_encoded,
                    tokenizer=tokenizer,
                    data_collator=data_collator)
        
        trainer.train()

        tokenizer.save_pretrained(f'saved/{model_name}')
        model.save_pretrained(f'saved/{model_name}')

        model_ckpt = model_ckpt.replace('klue-roberta-base', 'klue/roberta-base')

        foldNum += 1

print('finish')


----------------------------------------------------------------------------
klue/roberta-base

----------------------------------------------------------------------------
klue/roberta-base 1-Fold


Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifie

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

...

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.575800,0.416255,0.861918,0.861088
2,0.382200,0.368863,0.878313,0.877140
3,0.290700,0.339138,0.892941,0.891059
4,0.224500,0.353287,0.898292,0.897820
5,0.167500,0.351455,0.906882,0.906406
6,0.127800,0.394038,0.907962,0.906864
7,0.096800,0.428210,0.912920,0.912090
8,0.075900,0.451528,0.917387,0.916709
9,0.057600,0.481573,0.918368,0.918044
10,0.045700,0.493551,0.918859,0.918429


finish


### Logit Ensemble

In [12]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# torch.cuda.empty_cache()
# !nvidia-smi -L

In [13]:
# import gc
# gc.collect()

In [14]:
saved_models = os.listdir('saved')
saved_models.sort()

In [15]:
for e in saved_models:
    print(e)

bert-base-multilingual-uncased-tc-1
bert-base-multilingual-uncased-tc-2
bert-base-multilingual-uncased-tc-3
bert-base-multilingual-uncased-tc-4
bert-base-multilingual-uncased-tc-5
klue-roberta-base-tc-1
klue-roberta-base-tc-2
klue-roberta-base-tc-3
klue-roberta-base-tc-4
klue-roberta-base-tc-5
xlm-roberta-base-tc-1
xlm-roberta-base-tc-2
xlm-roberta-base-tc-3
xlm-roberta-base-tc-4
xlm-roberta-base-tc-5


In [16]:
final_logits = 0
for saved_model in saved_models:
    model = AutoModelForSequenceClassification.from_pretrained('saved/' + saved_model)
    tokenizer = AutoTokenizer.from_pretrained('saved/' + saved_model)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    
    data_test = datasets.load_dataset('csv', data_files=test_data_path)['train']
    data_test = data_test.remove_columns('index')
    encoded_test = data_test.map(tokenize, batched=True, batch_size=None)
    encoded_test = encoded_test.remove_columns('title')

    training_args = TrainingArguments(output_dir='test_log/' + saved_model,
                                        per_device_eval_batch_size=batch_size,
                                        disable_tqdm=False)

    trainer = Trainer(model=model,
                args=training_args,
                tokenizer=tokenizer,
                data_collator=data_collator)

    preds_output = trainer.predict(encoded_test)

    final_logits += preds_output.predictions

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-89859000f922f8a5/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

***** Running Prediction *****
  Num examples = 9131
  Batch size = 32


...

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Prediction *****
  Num examples = 9131
  Batch size = 32


loading configuration file saved/xlm-roberta-base-tc-5/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "saved/xlm-roberta-base-tc-5",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Prediction *****
  Num examples = 9131
  Batch size = 32


In [ ]:
final_logits.argmax(1)

In [ ]:
sub = pd.read_csv("/content/sample_submission.csv")
sub['topic_idx'] = final_logits.argmax(1)
sub.to_csv('final_submission.csv', index=False)

In [19]:
sub

,index,topic_idx
0,45654,0
1,45655,3
2,45656,2
3,45657,0
4,45658,3
...,...,...
9126,54780,3
9127,54781,2
9128,54782,3
9129,54783,2


,index,topic_idx
0,45654,0
1,45655,3
2,45656,2
3,45657,0
4,45658,3
...,...,...
9126,54780,3
9127,54781,2
9128,54782,3
9129,54783,2
